In [ ]:
import argparse
from sys import platform

from models import *  # set ONNX_EXPORT in models.py
from utils.datasets import *
from utils.utils import *

cfg = 'cfg/fashion.cfg'
data = 'data/fashion/fashion.data'
weights = 'weights/exp1/best.pt'

source = 'off_images/' 
#source = 'on_images/'

output = 'output_off'

img_size = 416
conf_thres = 0.5
nms_thres = 0.3
half = False
view_img = False
device=''

img_size = (320, 192) if ONNX_EXPORT else img_size
out, source, weights, half, view_img = output, source, weights, half, view_img
webcam = source == '0' or source.startswith('rtsp') or source.startswith('http') or source.endswith('.txt')
# Initialize
device = torch_utils.select_device(device='cpu' if ONNX_EXPORT else device)

model = Darknet(cfg, img_size)

if weights.endswith('.pt'):  # pytorch format
    model.load_state_dict(torch.load(weights, map_location=device)['model'])
else:  # darknet format
    _ = load_darknet_weights(model, weights)

model.to(device).eval()

In [ ]:
# Set Dataloader
vid_path, vid_writer = None, None
save_txt = True
save_img = True
dataset = LoadImages(source, img_size=img_size, half=half)

# Get classes and colors
classes = load_classes(parse_data_cfg(data)['names'])
colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(classes))]

In [ ]:
dataset.

In [ ]:
if os.path.exists(out):
    shutil.rmtree(out)  # delete output folder
os.makedirs(out)  # make new output folder

t0 = time.time()

no_list =[]
yes_list = []

for path, img, im0s, vid_cap in dataset:
    t = time.time()
    
    any_box = True
    
    
    # Get detections
    img = torch.from_numpy(img).to(device)
    if img.ndimension() == 3:
        img = img.unsqueeze(0)
    pred, _ = model(img)

    for i, det in enumerate(non_max_suppression(pred, conf_thres, nms_thres)):  # detections per image
        p, s, im0 = path, '', im0s
        img_width, img_height = img.shape[2:]
        
        save_path = str(Path(out) / Path(p).name.split('.')[0])
        if det is not None and len(det):
            boxes = len(det)
            
            for *xyxy, conf, _, cls in det:

                with open(save_path + '.txt', 'a') as file:
                    file.write(('%g ' * 7 + '\n') % (*xyxy, cls, img_width,img_height))
                
                if cls in (4,6,7,8): # 4 bag, 6 Headwear, 7 Acc, 8 Innerwear # 1차 제거
                    boxes -= 1
                    continue     
                    
                else: # 2차 제거 (박스 크기)
                    
                    x1,y1,x3,y3 = xyxy
                    
                    box_width = x3-x1
                    box_height = y3-y1
                            
                    spatial_ratio = (box_width * box_height) / (img_width * img_height)
                    
                    if spatial_ratio < 1 / 14:
                        boxes -= 1
                        continue
                         
            if boxes == 0:
                no_list.append(path)
            else:
                any_box = False
                yes_list.append(path)
        else:
            no_list.append(path)
        print('')

In [ ]:
dataset.count
len(no_list)

In [ ]:
print('Detect no box {:.2%}'.format(len(no_list) / dataset.count))
print('Detect any box {:.2%}'.format(len(yes_list) / dataset.count))

In [ ]:
import matplotlib.pyplot as plt
import PIL
import random

random.shuffle(no_list)

for no_detect_img in no_list[:100]:
    img = PIL.Image.open(no_detect_img)
    plt.figure()
    plt.imshow(img)
    plt.axis('off')


In [ ]:
random.shuffle(yes_list)

for yes_detect_img in yes_list[:100]:
    img = PIL.Image.open(yes_detect_img)

    plt.figure()
    plt.imshow(img)
    plt.axis('off')


In [ ]:
import shutil

for no_detect_img in no_list:
    des = 'no_detect/'+ no_detect_img.split('/')[-1]
    shutil.move(no_detect_img, des)

for yes_detect_img in yes_list:
    des = 'yes_detect/'+ yes_detect_img.split('/')[-1]
    shutil.move(yes_detect_img, des)

In [ ]:
for no_detect_img in no_list:
    des = 'no_detect_on/'+ no_detect_img.split('/')[-1]
    shutil.move(no_detect_img, des)

for yes_detect_img in yes_list:
    des = 'yes_detect_on/'+ yes_detect_img.split('/')[-1]
    shutil.move(yes_detect_img, des)

In [ ]:
###

source = 'yes_detect' # Screening 된 이미지들
out = 'output/'

if os.path.exists(out):
    shutil.rmtree(out)  # delete output folder
os.makedirs(out)  # make new output folder

# Set Dataloader
vid_path, vid_writer = None, None
save_txt = False
save_img = True
dataset = LoadImages(source, img_size=img_size, half=half)

# Get classes and colors
classes = load_classes(parse_data_cfg(data)['names'])
colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(classes))]

t0 = time.time()

for path, img, im0s, vid_cap in dataset:
    t = time.time()

    # Get detections
    img = torch.from_numpy(img).to(device)
    if img.ndimension() == 3:
        img = img.unsqueeze(0)
    pred, _ = model(img)

    for i, det in enumerate(non_max_suppression(pred, conf_thres, nms_thres)):  # detections per image
        if webcam:  # batch_size >= 1
            p, s, im0 = path[i], '%g: ' % i, im0s[i]
        else:
            p, s, im0 = path, '', im0s

        save_path = str(Path(out) / Path(p).name)
        s += '%gx%g ' % img.shape[2:]  # print string
        if det is not None and len(det):
            # Rescale boxes from img_size to im0 size
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()
                    
            # Print results
            for c in det[:, -1].unique():
                n = (det[:, -1] == c).sum()  # detections per class
                s += '%g %ss, ' % (n, classes[int(c)])  # add to string

            # Write results
            for *xyxy, conf, _, cls in det:
                
                if cls in (4,6,7,8): # 4 bag, 6 Headwear, 7 Acc, 8 Innerwear
                    continue
                    
                if save_txt:  # Write to file
                    with open(save_path + '.txt', 'a') as file:
                        file.write(('%g ' * 6 + '\n') % (*xyxy, cls, conf))
                        
                if save_img or view_img:  # Add bbox to image
                    label = '%s %.2f' % (classes[int(cls)], conf)
                    plot_one_box(xyxy, im0, label=label, color=colors[int(cls)])

        print('%sDone. (%.3fs)' % (s, time.time() - t))

        # Save results (image with detections)
        if save_img:
            if dataset.mode == 'images':
                cv2.imwrite(save_path, im0)
            else:
                if vid_path != save_path:  # new video
                    vid_path = save_path
                    if isinstance(vid_writer, cv2.VideoWriter):
                        vid_writer.release()  # release previous video writer

                    fps = vid_cap.get(cv2.CAP_PROP_FPS)
                    w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                    h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                    vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*opt.fourcc), fps, (w, h))
                vid_writer.write(im0)

if save_txt or save_img:
    print('Results saved to %s' % os.getcwd() + os.sep + out)
    if platform == 'darwin':  # MacOS
        os.system('open ' + out + ' ' + save_path)

print('Done. (%.3fs)' % (time.time() - t0))